In [1]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
import json
import torch
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
# Load dataset
dataset = load_dataset('fever', 'v1.0', split='train')
df = pd.DataFrame(dataset)[:100]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for fever contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fever
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/311431 [00:00<?, ? examples/s]

Generating labelled_dev split:   0%|          | 0/37566 [00:00<?, ? examples/s]

Generating unlabelled_dev split:   0%|          | 0/19998 [00:00<?, ? examples/s]

Generating unlabelled_test split:   0%|          | 0/19998 [00:00<?, ? examples/s]

Generating paper_dev split:   0%|          | 0/18999 [00:00<?, ? examples/s]

Generating paper_test split:   0%|          | 0/18567 [00:00<?, ? examples/s]

In [4]:
y = df.label
df_features = ['claim']
X = df[df_features]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [6]:
# BERT model setup
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(df['label'].unique()))


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [8]:
# Tokenize inputs and convert to tensors
X_train_texts = list(X_train['claim'])
X_test_texts = list(X_test['claim'])
y_train_labels = label_encoder.fit_transform(y_train)
y_test_labels = label_encoder.transform(y_test)


In [9]:
X_train_inputs = tokenizer(X_train_texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
X_test_inputs = tokenizer(X_test_texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

y_train_labels = torch.tensor(y_train_labels, dtype=torch.long)
y_test_labels = torch.tensor(y_test_labels, dtype=torch.long)


In [10]:
# Fine-tune the model
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


In [20]:
for epoch in range(2):  # Adjust the number of epochs as needed
    optimizer.zero_grad()
    outputs = model(**X_train_inputs, labels=y_train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

In [21]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(**X_test_inputs)
    test_predictions = torch.argmax(test_outputs.logits, dim=1)


In [22]:
# Convert predictions to numpy arrays
y_test_np = y_test_labels.numpy()
test_predictions_np = test_predictions.numpy()


In [23]:
from sklearn.metrics import precision_score, recall_score, classification_report


accuracy = torch.eq(test_predictions, y_test_labels).sum().item() / len(y_test_labels)
precision_micro = precision_score(y_test_np, test_predictions_np, average='micro')
precision_macro = precision_score(y_test_np, test_predictions_np, average='macro')
recall_micro = recall_score(y_test_np, test_predictions_np, average='micro')
recall_macro = recall_score(y_test_np, test_predictions_np, average='macro')
f1_micro = f1_score(y_test_np, test_predictions_np, average='micro')
f1_macro = f1_score(y_test_np, test_predictions_np, average='macro')

# Print evaluation statistics
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision_macro}")
print(f"Test Recall: {recall_macro}")
print(f"Test F1: {f1_macro}")
print("Classification Report:\n", classification_report(y_test_np, test_predictions_np))

Test Accuracy: 0.64
Test Precision: 0.25396825396825395
Test Recall: 0.2962962962962963
Test F1: 0.2735042735042735
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         6
           2       0.76      0.89      0.82        18

    accuracy                           0.64        25
   macro avg       0.25      0.30      0.27        25
weighted avg       0.55      0.64      0.59        25



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13